# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.


In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

# **11_EYE**

In [ ]:
df_11_EYE = pd.read_csv('data/STData/11/11_EYE.csv')

In [ ]:
df_11_EYE.head()

In [ ]:
df_11_EYE.shape

In [ ]:
df_11_EYE.columns

In [ ]:
df_11_EYE.info()

In [ ]:
df_11_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_11_EYE['QuestionKey'].unique()

In [ ]:
df_11_EYE['Timestamp'] = pd.to_datetime(df_11_EYE['Timestamp'])

In [ ]:
df_11_EYE.head(3)

In [ ]:
df_11_EYE['QuestionKey'] = df_11_EYE['QuestionKey'].fillna('None')

In [ ]:
df_11_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_11_EYE.isnull().sum()

In [ ]:
df_11_EYE.dropna(inplace=True)

In [ ]:
df_11_EYE.head()

In [ ]:
df_11_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_11_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_11_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_11_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_11_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_11_EYE['ET_ValidityRight'].unique()

In [ ]:
df_11_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_11_EYE['ET_ValidityLeft'].value_counts().index, y=df_11_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_11_EYE['ET_ValidityRight'].value_counts().index, y=df_11_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_11_EYE['ET_ValidityLeft'] = df_11_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_11_EYE['ET_ValidityRight'] = df_11_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_11_EYE.head(3)

In [ ]:
df_11_EYE.describe()

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1].shape[0] / df_11_EYE.shape[0]

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1].shape[0] / df_11_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape[0] / df_11_EYE.shape[0]

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape[0] / df_11_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_11_EYE['ET_PupilLeft_validity'] = df_11_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_11_EYE['ET_PupilRight_validity'] = df_11_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_11_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_11_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_11_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_11_EYE['ET_PupilLeft_validity'] = df_11_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_11_EYE['ET_PupilRight_validity'] = df_11_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_11_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_11_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_11_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_11_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_11_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_11_EYE['ET_PupilLeft_validity'] = df_11_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_11_EYE['ET_PupilRight_validity'] = df_11_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_11_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_11_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_11_EYE['Timestamp'], df_11_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_11_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_11_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_11_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_11_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_11_EYE[col] = df_11_EYE[col].fillna(df_11_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_11_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_11_EYE['Timestamp'], df_11_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_11_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_11_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_11_EYE)
plt.show()

# **12_EYE**

In [ ]:
df_12_EYE = pd.read_csv('data/STData/12/12_EYE.csv')

In [ ]:
df_12_EYE.head()

In [ ]:
df_12_EYE.shape

In [ ]:
df_12_EYE.columns

In [ ]:
df_12_EYE.info()

In [ ]:
df_12_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_12_EYE['QuestionKey'].unique()

In [ ]:
df_12_EYE['Timestamp'] = pd.to_datetime(df_12_EYE['Timestamp'])

In [ ]:
df_12_EYE.head(3)

In [ ]:
df_12_EYE['QuestionKey'] = df_12_EYE['QuestionKey'].fillna('None')

In [ ]:
df_12_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_12_EYE.isnull().sum()

In [ ]:
df_12_EYE.dropna(inplace=True)

In [ ]:
df_12_EYE.head()

In [ ]:
df_12_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_12_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_12_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_12_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_12_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_12_EYE['ET_ValidityRight'].unique()

In [ ]:
df_12_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_12_EYE['ET_ValidityLeft'].value_counts().index, y=df_12_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_12_EYE['ET_ValidityRight'].value_counts().index, y=df_12_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_12_EYE['ET_ValidityLeft'] = df_12_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_12_EYE['ET_ValidityRight'] = df_12_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_12_EYE.head(3)

In [ ]:
df_12_EYE.describe()

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1].shape[0] / df_12_EYE.shape[0]

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1].shape[0] / df_12_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape[0] / df_12_EYE.shape[0]

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape[0] / df_12_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_12_EYE['ET_PupilLeft_validity'] = df_12_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_12_EYE['ET_PupilRight_validity'] = df_12_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_12_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_12_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_12_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_12_EYE['ET_PupilLeft_validity'] = df_12_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_12_EYE['ET_PupilRight_validity'] = df_12_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_12_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_12_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_12_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_12_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_12_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_12_EYE['ET_PupilLeft_validity'] = df_12_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_12_EYE['ET_PupilRight_validity'] = df_12_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_12_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_12_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_12_EYE['Timestamp'], df_12_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_12_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_12_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_12_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_12_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_12_EYE[col] = df_12_EYE[col].fillna(df_12_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_12_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_12_EYE['Timestamp'], df_12_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_12_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_12_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_12_EYE)
plt.show()

# **13_EYE**

In [ ]:
df_13_EYE = pd.read_csv('data/STData/13/13_EYE.csv')

In [ ]:
df_13_EYE.head()

In [ ]:
df_13_EYE.shape

In [ ]:
df_13_EYE.columns

In [ ]:
df_13_EYE.info()

In [ ]:
df_13_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_13_EYE['QuestionKey'].unique()

In [ ]:
df_13_EYE['Timestamp'] = pd.to_datetime(df_13_EYE['Timestamp'])

In [ ]:
df_13_EYE.head(3)

In [ ]:
df_13_EYE['QuestionKey'] = df_13_EYE['QuestionKey'].fillna('None')

In [ ]:
df_13_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_13_EYE.isnull().sum()

In [ ]:
df_13_EYE.dropna(inplace=True)

In [ ]:
df_13_EYE.head()

In [ ]:
df_13_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_13_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_13_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_13_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_13_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_13_EYE['ET_ValidityRight'].unique()

In [ ]:
df_13_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_13_EYE['ET_ValidityLeft'].value_counts().index, y=df_13_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_13_EYE['ET_ValidityRight'].value_counts().index, y=df_13_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_13_EYE['ET_ValidityLeft'] = df_13_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_13_EYE['ET_ValidityRight'] = df_13_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_13_EYE.head(3)

In [ ]:
df_13_EYE.describe()

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1].shape[0] / df_13_EYE.shape[0]

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1].shape[0] / df_13_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape[0] / df_13_EYE.shape[0]

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape[0] / df_13_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_13_EYE['ET_PupilLeft_validity'] = df_13_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_13_EYE['ET_PupilRight_validity'] = df_13_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_13_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_13_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_13_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_13_EYE['ET_PupilLeft_validity'] = df_13_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_13_EYE['ET_PupilRight_validity'] = df_13_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_13_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_13_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_13_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_13_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_13_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_13_EYE['ET_PupilLeft_validity'] = df_13_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_13_EYE['ET_PupilRight_validity'] = df_13_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_13_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_13_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_13_EYE['Timestamp'], df_13_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_13_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_13_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_13_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_13_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_13_EYE[col] = df_13_EYE[col].fillna(df_13_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_13_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_13_EYE['Timestamp'], df_13_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_13_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_13_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_13_EYE)
plt.show()

# **14_EYE**

In [ ]:
df_14_EYE = pd.read_csv('data/STData/14/14_EYE.csv')

In [ ]:
df_14_EYE.head()

In [ ]:
df_14_EYE.shape

In [ ]:
df_14_EYE.columns

In [ ]:
df_14_EYE.info()

In [ ]:
df_14_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_14_EYE['QuestionKey'].unique()

In [ ]:
df_14_EYE['Timestamp'] = pd.to_datetime(df_14_EYE['Timestamp'])

In [ ]:
df_14_EYE.head(3)

In [ ]:
df_14_EYE['QuestionKey'] = df_14_EYE['QuestionKey'].fillna('None')

In [ ]:
df_14_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_14_EYE.isnull().sum()

In [ ]:
df_14_EYE.dropna(inplace=True)

In [ ]:
df_14_EYE.head()

In [ ]:
df_14_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_14_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_14_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_14_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_14_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_14_EYE['ET_ValidityRight'].unique()

In [ ]:
df_14_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_14_EYE['ET_ValidityLeft'].value_counts().index, y=df_14_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_14_EYE['ET_ValidityRight'].value_counts().index, y=df_14_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_14_EYE['ET_ValidityLeft'] = df_14_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_14_EYE['ET_ValidityRight'] = df_14_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_14_EYE.head(3)

In [ ]:
df_14_EYE.describe()

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1].shape[0] / df_14_EYE.shape[0]

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1].shape[0] / df_14_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape[0] / df_14_EYE.shape[0]

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape[0] / df_14_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_14_EYE['ET_PupilLeft_validity'] = df_14_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_14_EYE['ET_PupilRight_validity'] = df_14_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_14_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_14_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_14_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_14_EYE['ET_PupilLeft_validity'] = df_14_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_14_EYE['ET_PupilRight_validity'] = df_14_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_14_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_14_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_14_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_14_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_14_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_14_EYE['ET_PupilLeft_validity'] = df_14_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_14_EYE['ET_PupilRight_validity'] = df_14_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_14_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_14_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_14_EYE['Timestamp'], df_14_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_14_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_14_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_14_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_14_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_14_EYE[col] = df_14_EYE[col].fillna(df_14_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_14_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_14_EYE['Timestamp'], df_14_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_14_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_14_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_14_EYE)
plt.show()

# **15_EYE**

In [ ]:
df_15_EYE = pd.read_csv('data/STData/15/15_EYE.csv')

In [ ]:
df_15_EYE.head()

In [ ]:
df_15_EYE.shape

In [ ]:
df_15_EYE.columns

In [ ]:
df_15_EYE.info()

In [ ]:
df_15_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_15_EYE['QuestionKey'].unique()

In [ ]:
df_15_EYE['Timestamp'] = pd.to_datetime(df_15_EYE['Timestamp'])

In [ ]:
df_15_EYE.head(3)

In [ ]:
df_15_EYE['QuestionKey'] = df_15_EYE['QuestionKey'].fillna('None')

In [ ]:
df_15_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_15_EYE.isnull().sum()

In [ ]:
df_15_EYE.dropna(inplace=True)

In [ ]:
df_15_EYE.head()

In [ ]:
df_15_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_15_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_15_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_15_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_15_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_15_EYE['ET_ValidityRight'].unique()

In [ ]:
df_15_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_15_EYE['ET_ValidityLeft'].value_counts().index, y=df_15_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_15_EYE['ET_ValidityRight'].value_counts().index, y=df_15_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_15_EYE['ET_ValidityLeft'] = df_15_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_15_EYE['ET_ValidityRight'] = df_15_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_15_EYE.head(3)

In [ ]:
df_15_EYE.describe()

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1].shape[0] / df_15_EYE.shape[0]

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1].shape[0] / df_15_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape[0] / df_15_EYE.shape[0]

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape[0] / df_15_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_15_EYE['ET_PupilLeft_validity'] = df_15_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_15_EYE['ET_PupilRight_validity'] = df_15_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_15_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_15_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_15_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_15_EYE['ET_PupilLeft_validity'] = df_15_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_15_EYE['ET_PupilRight_validity'] = df_15_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_15_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_15_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_15_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_15_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_15_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_15_EYE['ET_PupilLeft_validity'] = df_15_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_15_EYE['ET_PupilRight_validity'] = df_15_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_15_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_15_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_15_EYE['Timestamp'], df_15_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_15_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_15_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_15_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_15_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_15_EYE[col] = df_15_EYE[col].fillna(df_15_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_15_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_15_EYE['Timestamp'], df_15_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_15_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_15_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_15_EYE)
plt.show()

# **16_EYE**

In [ ]:
df_16_EYE = pd.read_csv('data/STData/16/16_EYE.csv')

In [ ]:
df_16_EYE.head()

In [ ]:
df_16_EYE.shape

In [ ]:
df_16_EYE.columns

In [ ]:
df_16_EYE.info()

In [ ]:
df_16_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_16_EYE['QuestionKey'].unique()

In [ ]:
df_16_EYE['Timestamp'] = pd.to_datetime(df_16_EYE['Timestamp'])

In [ ]:
df_16_EYE.head(3)

In [ ]:
df_16_EYE['QuestionKey'] = df_16_EYE['QuestionKey'].fillna('None')

In [ ]:
df_16_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_16_EYE.isnull().sum()

In [ ]:
df_16_EYE.dropna(inplace=True)

In [ ]:
df_16_EYE.head()

In [ ]:
df_16_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_16_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_16_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_16_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_16_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_16_EYE['ET_ValidityRight'].unique()

In [ ]:
df_16_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_16_EYE['ET_ValidityLeft'].value_counts().index, y=df_16_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_16_EYE['ET_ValidityRight'].value_counts().index, y=df_16_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_16_EYE['ET_ValidityLeft'] = df_16_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_16_EYE['ET_ValidityRight'] = df_16_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_16_EYE.head(3)

In [ ]:
df_16_EYE.describe()

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1].shape[0] / df_16_EYE.shape[0]

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1].shape[0] / df_16_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape[0] / df_16_EYE.shape[0]

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape[0] / df_16_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_16_EYE['ET_PupilLeft_validity'] = df_16_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_16_EYE['ET_PupilRight_validity'] = df_16_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_16_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_16_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_16_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_16_EYE['ET_PupilLeft_validity'] = df_16_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_16_EYE['ET_PupilRight_validity'] = df_16_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_16_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_16_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_16_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_16_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_16_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_16_EYE['ET_PupilLeft_validity'] = df_16_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_16_EYE['ET_PupilRight_validity'] = df_16_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_16_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_16_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_16_EYE['Timestamp'], df_16_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_16_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_16_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_16_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_16_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_16_EYE[col] = df_16_EYE[col].fillna(df_16_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_16_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_16_EYE['Timestamp'], df_16_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_16_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_16_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_16_EYE)
plt.show()

# **17_EYE**

In [ ]:
df_17_EYE = pd.read_csv('data/STData/17/17_EYE.csv')

In [ ]:
df_17_EYE.head()

In [ ]:
df_17_EYE.shape

In [ ]:
df_17_EYE.columns

In [ ]:
df_17_EYE.info()

In [ ]:
df_17_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_17_EYE['QuestionKey'].unique()

In [ ]:
df_17_EYE['Timestamp'] = pd.to_datetime(df_17_EYE['Timestamp'])

In [ ]:
df_17_EYE.head(3)

In [ ]:
df_17_EYE['QuestionKey'] = df_17_EYE['QuestionKey'].fillna('None')

In [ ]:
df_17_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_17_EYE.isnull().sum()

In [ ]:
df_17_EYE.dropna(inplace=True)

In [ ]:
df_17_EYE.head()

In [ ]:
df_17_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_17_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_17_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_17_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_17_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_17_EYE['ET_ValidityRight'].unique()

In [ ]:
df_17_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_17_EYE['ET_ValidityLeft'].value_counts().index, y=df_17_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_17_EYE['ET_ValidityRight'].value_counts().index, y=df_17_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_17_EYE['ET_ValidityLeft'] = df_17_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_17_EYE['ET_ValidityRight'] = df_17_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_17_EYE.head(3)

In [ ]:
df_17_EYE.describe()

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1].shape[0] / df_17_EYE.shape[0]

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1].shape[0] / df_17_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape[0] / df_17_EYE.shape[0]

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape[0] / df_17_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_17_EYE['ET_PupilLeft_validity'] = df_17_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_17_EYE['ET_PupilRight_validity'] = df_17_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_17_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_17_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_17_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_17_EYE['ET_PupilLeft_validity'] = df_17_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_17_EYE['ET_PupilRight_validity'] = df_17_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_17_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_17_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_17_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_17_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_17_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_17_EYE['ET_PupilLeft_validity'] = df_17_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_17_EYE['ET_PupilRight_validity'] = df_17_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_17_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_17_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_17_EYE['Timestamp'], df_17_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_17_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_17_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_17_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_17_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_17_EYE[col] = df_17_EYE[col].fillna(df_17_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_17_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_17_EYE['Timestamp'], df_17_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_17_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_17_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_17_EYE)
plt.show()

# **18_EYE**

In [ ]:
df_18_EYE = pd.read_csv('data/STData/18/18_EYE.csv')

In [ ]:
df_18_EYE.head()

In [ ]:
df_18_EYE.shape

In [ ]:
df_18_EYE.columns

In [ ]:
df_18_EYE.info()

In [ ]:
df_18_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_18_EYE['QuestionKey'].unique()

In [ ]:
df_18_EYE['Timestamp'] = pd.to_datetime(df_18_EYE['Timestamp'])

In [ ]:
df_18_EYE.head(3)

In [ ]:
df_18_EYE['QuestionKey'] = df_18_EYE['QuestionKey'].fillna('None')

In [ ]:
df_18_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_18_EYE.isnull().sum()

In [ ]:
df_18_EYE.dropna(inplace=True)

In [ ]:
df_18_EYE.head()

In [ ]:
df_18_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_18_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_18_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_18_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_18_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_18_EYE['ET_ValidityRight'].unique()

In [ ]:
df_18_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_18_EYE['ET_ValidityLeft'].value_counts().index, y=df_18_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_18_EYE['ET_ValidityRight'].value_counts().index, y=df_18_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_18_EYE['ET_ValidityLeft'] = df_18_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_18_EYE['ET_ValidityRight'] = df_18_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_18_EYE.head(3)

In [ ]:
df_18_EYE.describe()

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1].shape[0] / df_18_EYE.shape[0]

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1].shape[0] / df_18_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape[0] / df_18_EYE.shape[0]

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape[0] / df_18_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_18_EYE['ET_PupilLeft_validity'] = df_18_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_18_EYE['ET_PupilRight_validity'] = df_18_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_18_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_18_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_18_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_18_EYE['ET_PupilLeft_validity'] = df_18_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_18_EYE['ET_PupilRight_validity'] = df_18_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_18_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_18_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_18_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_18_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_18_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_18_EYE['ET_PupilLeft_validity'] = df_18_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_18_EYE['ET_PupilRight_validity'] = df_18_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_18_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_18_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_18_EYE['Timestamp'], df_18_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_18_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_18_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_18_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_18_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_18_EYE[col] = df_18_EYE[col].fillna(df_18_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_18_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_18_EYE['Timestamp'], df_18_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_18_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_18_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_18_EYE)
plt.show()

# **19_EYE**

In [ ]:
df_19_EYE = pd.read_csv('data/STData/19/19_EYE.csv')

In [ ]:
df_19_EYE.head()

In [ ]:
df_19_EYE.shape

In [ ]:
df_19_EYE.columns

In [ ]:
df_19_EYE.info()

In [ ]:
df_19_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_19_EYE['QuestionKey'].unique()

In [ ]:
df_19_EYE['Timestamp'] = pd.to_datetime(df_19_EYE['Timestamp'])

In [ ]:
df_19_EYE.head(3)

In [ ]:
df_19_EYE['QuestionKey'] = df_19_EYE['QuestionKey'].fillna('None')

In [ ]:
df_19_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_19_EYE.isnull().sum()

In [ ]:
df_19_EYE.dropna(inplace=True)

In [ ]:
df_19_EYE.head()

In [ ]:
df_19_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_19_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_19_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_19_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_19_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_19_EYE['ET_ValidityRight'].unique()

In [ ]:
df_19_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_19_EYE['ET_ValidityLeft'].value_counts().index, y=df_19_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_19_EYE['ET_ValidityRight'].value_counts().index, y=df_19_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_19_EYE['ET_ValidityLeft'] = df_19_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_19_EYE['ET_ValidityRight'] = df_19_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_19_EYE.head(3)

In [ ]:
df_19_EYE.describe()

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1].shape[0] / df_19_EYE.shape[0]

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1].shape[0] / df_19_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape[0] / df_19_EYE.shape[0]

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape[0] / df_19_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_19_EYE['ET_PupilLeft_validity'] = df_19_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_19_EYE['ET_PupilRight_validity'] = df_19_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_19_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_19_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_19_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_19_EYE['ET_PupilLeft_validity'] = df_19_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_19_EYE['ET_PupilRight_validity'] = df_19_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_19_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_19_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_19_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_19_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_19_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_19_EYE['ET_PupilLeft_validity'] = df_19_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_19_EYE['ET_PupilRight_validity'] = df_19_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_19_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_19_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_19_EYE['Timestamp'], df_19_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_19_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_19_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_19_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_19_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_19_EYE[col] = df_19_EYE[col].fillna(df_19_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_19_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_19_EYE['Timestamp'], df_19_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_19_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_19_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_19_EYE)
plt.show()

# **20_EYE**

In [ ]:
df_20_EYE = pd.read_csv('data/STData/20/20_EYE.csv')

In [ ]:
df_20_EYE.head()

In [ ]:
df_20_EYE.shape

In [ ]:
df_20_EYE.columns

In [ ]:
df_20_EYE.info()

In [ ]:
df_20_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_20_EYE['QuestionKey'].unique()

In [ ]:
df_20_EYE['Timestamp'] = pd.to_datetime(df_20_EYE['Timestamp'])

In [ ]:
df_20_EYE.head(3)

In [ ]:
df_20_EYE['QuestionKey'] = df_20_EYE['QuestionKey'].fillna('None')

In [ ]:
df_20_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_20_EYE.isnull().sum()

In [ ]:
df_20_EYE.dropna(inplace=True)

In [ ]:
df_20_EYE.head()

In [ ]:
df_20_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_20_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_20_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_20_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_20_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_20_EYE['ET_ValidityRight'].unique()

In [ ]:
df_20_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_20_EYE['ET_ValidityLeft'].value_counts().index, y=df_20_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_20_EYE['ET_ValidityRight'].value_counts().index, y=df_20_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_20_EYE['ET_ValidityLeft'] = df_20_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_20_EYE['ET_ValidityRight'] = df_20_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_20_EYE.head(3)

In [ ]:
df_20_EYE.describe()

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1].shape[0] / df_20_EYE.shape[0]

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1].shape[0] / df_20_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape[0] / df_20_EYE.shape[0]

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape[0] / df_20_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_20_EYE['ET_PupilLeft_validity'] = df_20_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_20_EYE['ET_PupilRight_validity'] = df_20_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_20_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_20_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_20_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_20_EYE['ET_PupilLeft_validity'] = df_20_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_20_EYE['ET_PupilRight_validity'] = df_20_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_20_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_20_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_20_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_20_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_20_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_20_EYE['ET_PupilLeft_validity'] = df_20_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_20_EYE['ET_PupilRight_validity'] = df_20_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_20_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_20_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_20_EYE['Timestamp'], df_20_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_20_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_20_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_20_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_20_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_20_EYE[col] = df_20_EYE[col].fillna(df_20_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_20_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_20_EYE['Timestamp'], df_20_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_20_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_20_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_20_EYE)
plt.show()